In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
import math
import word_dist_script as ws
import pandas as pd

In [2]:
twitEmbs = ws.load_twitter_embs()

loading finished


In [3]:
load_word2vec = True

if load_word2vec:
    word2vec = ws.load_word2vec_embs()


Loading word2vec model...
This can take several minutes...


In [4]:
# every_emb = ws.get_every_embeddings(word2vec)
# clf = KMeans(n_clusters=5, random_state=0)

# clf_kmoids = KMeans(n_clusters=5, random_state=0, metric='precomputed', method='pam')

# clf_kmoids.fit(every_emb)

# pred_emb = clf_kmoids.predict(every_emb)

In [5]:
every_dist = ws.get_every_distance(word2vec)
import random
random.seed(4012)

clf_kmoids = KMedoids(n_clusters=6, random_state=4012, metric='precomputed', method='pam')


clf_kmoids.fit(every_dist)

pred_dist = clf_kmoids.predict(every_dist)

In [6]:
pred_dist

array([0, 5, 2, 4, 3, 5, 0, 2, 3, 1, 1, 1, 2, 3, 2, 5, 3, 0, 5, 0, 5, 5,
       0, 1, 5, 1, 1, 2, 5, 2, 4, 4, 5, 3, 0, 4, 1], dtype=int64)

In [8]:
sorted_dist = ws.sort_zip_labels(pred_dist)
groups_dist = ws.group_by_cluster(sorted_dist, n_clusters=6)

# sorted_emb = ws.sort_zip_labels(pred_emb)
# groups_emb = ws.group_by_cluster(sorted_emb)

In [9]:
for i, group in enumerate(groups_dist):
    print(i, group)

0 ['journal', 'book', 'magazine', 'misc', 'Organisation', 'theory']
1 ['country', 'discipline', 'election', 'person', 'politics', 'politician', 'writer']
2 ['algorithm', 'chemical', 'enzyme', 'field', 'product', 'protein']
3 ['award', 'conference', 'event', 'location', 'task']
4 ['astronomer', 'researcher', 'scientist', 'university']
5 ['album', 'band', 'genre', 'metrics', 'artist', 'instrument', 'poem', 'java', 'song']


In [ ]:
# for i, group in enumerate(groups_emb):
#     print(i, group)

In [ ]:
for group in groups_dist:
    ## find distance to centroid
    label = ws.find_label_for_cluster(word2vec, group)
    print(label, group)

genre ['album', 'band', 'genre', 'location', 'metrics', 'artist', 'instrument', 'poem', 'java', 'song']
magazine ['journal', 'award', 'book', 'conference', 'event', 'magazine', 'misc', 'Organisation', 'theory']
politician ['country', 'discipline', 'election', 'person', 'politics', 'politician', 'writer']
enzyme ['algorithm', 'chemical', 'enzyme', 'field', 'product', 'protein']
astronomer ['astronomer', 'researcher', 'scientist', 'task', 'university']


In [ ]:
# for group in groups_emb:
#     ## find distance to centroid
#     label = ws.find_label_for_cluster_emb(word2vec, group)
#     print(label, group)

In [ ]:
# for group in groups_emb:
#     # find most similar word
#     centroid = ws.find_vector_for_cluster(word2vec, group)
#     most_sim = word2vec.most_similar(centroid)[0]
#     print(most_sim)

In [ ]:
entity2cluster_dist = ws.create_entity2cluster_dict(pred_dist)
entity2cluster_emb = ws.create_entity2cluster_dict(pred_emb)

# cluster2label_dist = ws.create_cluster2label_dict(word2vec, pred_dist)
# cluster2label_emb = ws.create_cluster2label_dict(word2vec, pred_emb)


NameError: name 'pred_emb' is not defined

## Mapping entities

In [ ]:
## load csv
df = pd.read_csv('../data/category_mappings.csv')
## get entity names
entities = df["entity_name"]

In [ ]:
def find_problematic_words(word2vec_model, entities):
    PROBLEMATIC_WORDS = []
    for entity in entities:
        try:
            word2vec_model[entity]
        except:
            PROBLEMATIC_WORDS.append(entity)
    return PROBLEMATIC_WORDS

In [ ]:
## find the problematic words
problem_words = find_problematic_words(word2vec_model=word2vec, entities=entities)
problem2work = {"musicalartist": "artist", "organisation": "Organisation", "politicalparty": "politics", "academicjournal": "journal", "chemicalcompound": "chemical", "chemicalelement": "chemical", "astronomicalobject": "astronomer", "musicgenre": "genre", "literarygenre": "genre", "programlang": "java", "musicalinstrument": "instrument"  }

In [ ]:
## map the problematic entities to working ones
working = entities.map(lambda entity: problem2work[entity] if entity in problem2work else entity)

In [ ]:
## map the words to their cluster(word) label
mapped_words_dist = working.map(lambda entity: cluster2label_dist[entity2cluster_dist[entity]])
# mapped_words_emb = working.map(lambda entity: cluster2label_emb[entity2cluster_emb[entity]])

In [ ]:
## add the new columns to the dataframe
df["word_dist"] = mapped_words_dist
# df["word_emb"] = mapped_words_emb

In [ ]:
## write to csv
df.to_csv('../data/category_mappings.csv', index=False)

In [ ]:
file = pd.read_csv("../data/category_mappings.csv", index_col=0)

In [ ]:
file["ours"]["politicalparty"]

'organisation'

In [ ]:
df_to_use = file["ours"]